In [1]:
import pandas as pd

contracts19 = pd.read_csv('Data/2019 Contracts.csv', low_memory=False)
contracts20 = pd.read_csv('Data/2020 Contracts.csv', low_memory=False)
contracts21 = pd.read_csv('Data/2021 Contracts.csv', low_memory=False)
contracts22 = pd.read_csv('Data/2022 Contracts.csv', low_memory=False)
contracts23 = pd.read_csv('Data/2023 Contracts.csv', low_memory=False)

contracts = pd.concat([contracts19, contracts20, contracts21, contracts22, contracts23], ignore_index=True)

In [2]:
contracts = contracts[contracts['type_of_set_aside'] != 'VETERAN SET ASIDE']
contracts['naics_code'] = contracts['naics_code'].fillna(0).astype(int).astype(str)

def set_asides(x):
    if x['type_of_set_aside'] in ['HUBZONE SET-ASIDE', 'HUBZONE SOLE SOURCE']:
        x['type_of_set_aside'] = "HUBZone"

    elif x['type_of_set_aside'] in ['SMALL BUSINESS SET ASIDE - PARTIAL', 'SMALL BUSINESS SET ASIDE - TOTAL']:
        x['type_of_set_aside'] = "Small Business"

    return(x)

contracts = contracts.apply(set_asides, axis=1)

contracts['total_obligated_amount'] = contracts['total_obligated_amount'].fillna(0)

contracts_to_keep = [
    "contract_award_unique_key", "total_obligated_amount", "current_total_value_of_award", "potential_total_value_of_award",
    "award_base_action_date", "award_base_action_date_fiscal_year", "period_of_performance_current_end_date", "awarding_agency_name",
    "awarding_office_name", "funding_agency_name", "funding_office_name", "recipient_uei", "recipient_name", "recipient_city_name",
    "recipient_county_name", "recipient_state_name", "primary_place_of_performance_city_name", "primary_place_of_performance_state_name",
    "award_or_idv_flag", "award_type", "product_or_service_code", "product_or_service_code_description",
    "naics_code", "naics_description", "type_of_set_aside", "local_area_set_aside", "multi_year_contract", 
    "contracting_officers_determination_of_business_size"
]

contracts = contracts[contracts_to_keep]

# Q1. A breakdown of Hubzone awards nationally by NAICS (last 5 years).  Just need top 10. 

In [ ]:
# Top 10 NAICS codes by awarding amount
import pandasql as psql

hubzones = pd.read_csv('Data/HUBZone Set Asides FY19-23.csv', low_memory=False)

query = """
SELECT naics_description AS Description, naics_code AS NAICS, SUM(total_obligated_amount) AS "Total Awarded", COUNT(*) AS "# of Contracts"
FROM hubzones
GROUP BY naics_code, naics_description
ORDER BY "Total Awarded" DESC
LIMIT 10
"""

naicsGroups = psql.sqldf(query, locals())

# naicsGroups.to_excel('Deliverables/HUBZone Awards by NAICS.xlsx')
naicsGroups

# Q2. Businesses that are registered in SAM as well as have won at least one contract for Dodge County.

In [17]:
dodgeContracts = pd.read_csv('Data/Dodge Contracts FY19-23.csv')

def get_address(x):
    x['address'] = f'{x['recipient_address_line_1']}, {x['recipient_city_name']}, {x['recipient_state_name']} {x['recipient_zip_4_code']}'

    return x

dodgeContracts = dodgeContracts.apply(get_address, axis=1)

In [18]:
import pandasql as psql

query= """
SELECT 
    recipient_name AS "Company Name", 
    recipient_uei AS UEI, 
    naics_code AS "Primary NAICS", 
    naics_description AS "NAICS Description", 
    SUM(total_obligated_amount) as "Total Awarded",
    COUNT(*) AS "Number of Contracts Awarded",
    address AS Address
FROM dodgeContracts
GROUP BY UEI
ORDER BY "Total Awarded" DESC
"""

sumDodgeContracts = psql.sqldf(query, locals())

sumDodgeContracts.to_excel('Deliverables/Dodge County Contracts.xlsx')
sumDodgeContracts

,Company Name,UEI,Primary NAICS,NAICS Description,Total Awarded,Number of Contracts Awarded,Address
0,KELLY GROUP INC,C8GHA13W6135,332311,PREFABRICATED METAL BUILDING AND COMPONENT MAN...,2319478.00,14,"220 N H ST, FREMONT, NEBRASKA 680254982"
1,READYTECH-GO RADIOLOGY LLC,HL1JHW95E133,561320,TEMPORARY HELP SERVICES,2168661.75,28,"1005 E 23RD ST STE 200, FREMONT, NEBRASKA 6802..."
2,T & L FLETCHER,Z165SKNS8MF5,561720,JANITORIAL SERVICES,410841.24,5,"640 N CHRISTY ST, FREMONT, NEBRASKA 680254615"
3,D & E FLETCHER,XT6KHRN5VTL4,561720,JANITORIAL SERVICES,74385.00,1,"905 N PIERCE ST, FREMONT, NEBRASKA 680253949"
4,JAYHAWK BOXES INC,JBBUUVK4HJS8,322211,CORRUGATED AND SOLID FIBER BOX MANUFACTURING,41002.64,1,"1150 S UNION ST, FREMONT, NEBRASKA 680256137"
5,BUTLER MACHINERY CO,KUZEFXHH8WA9,811310,COMMERCIAL AND INDUSTRIAL MACHINERY AND EQUIPM...,7357.05,1,"2831 N CR 20 AVE, FREMONT, NEBRASKA 680257328"
6,HOOPER TELEPHONE CO,EPS6PCEAVAP9,517311,WIRED TELECOMMUNICATIONS CARRIERS,4637.66,2,"101 N ELM ST, HOOPER, NEBRASKA 68031"


# Q3. Top 10 agencies awarding contracts to small businesses and hubzone businesses – nationally and regionally – by set aside as well as by socio-type.

## By Set-Aside

In [ ]:
# Top agencies by HUBZone set-aside (nationally)

query = """
SELECT awarding_agency_name AS "Awarding Agency", type_of_set_aside AS "Set Aside Type", 
    SUM(total_obligated_amount) AS "Total Awarded", COUNT(*) AS "# of Contracts"
FROM contracts
WHERE "Set Aside Type" == "HUBZone"
GROUP BY "Awarding Agency", "Set Aside Type"
ORDER BY "Total Awarded" DESC
"""

agency_hubzone = psql.sqldf(query, locals())

# agency_hubzone.to_excel('Deliverables/National HUBZone Agencies.xlsx')
agency_hubzone

In [ ]:
# Top agencies by Small Business set-aside (nationally)

query = """
SELECT awarding_agency_name AS "Awarding Agency", type_of_set_aside AS "Set Aside Type", 
    SUM(total_obligated_amount) AS "Total Awarded", COUNT(*) AS "# of Contracts"
FROM contracts
WHERE "Set Aside Type" == "Small Business"
GROUP BY "Awarding Agency", "Set Aside Type"
ORDER BY "Total Awarded" DESC
"""

agency_sb = psql.sqldf(query, locals())

# agency_sb.to_excel('Deliverables/National Small Business Agencies.xlsx')
agency_sb

In [ ]:
# Top agencies by HUBZone set-aside (regionally)

query = """
SELECT awarding_agency_name AS "Awarding Agency", type_of_set_aside AS "Set Aside", 
    SUM(total_obligated_amount) AS "Total Awarded", COUNT(*) AS "# of Contracts"
FROM contracts
WHERE "Set Aside" == "HUBZone"
    AND primary_place_of_performance_state_name IN ('NEBRASKA', 'IOWA', 'KANSAS', 'MISSOURI')
GROUP BY "Awarding Agency", "Set Aside Type"
ORDER BY "Total Awarded" DESC
"""

agency_hubzone_regional = psql.sqldf(query, locals())

# agency_hubzone_regional.to_excel('Deliverables/Regional HUBZone Agencies.xlsx')
agency_hubzone_regional

In [ ]:
# Top agencies by Small Business set-aside (regionally)

query = """
SELECT awarding_agency_name AS "Awarding Agency", type_of_set_aside AS "Set Aside", 
    SUM(total_obligated_amount) AS "Total Awarded", COUNT(*) AS "# of Contracts"
FROM contracts
WHERE "Set Aside" == "Small Business"
    AND primary_place_of_performance_state_name IN ('NEBRASKA', 'IOWA', 'KANSAS', 'MISSOURI')
GROUP BY "Awarding Agency", "Set Aside Type"
ORDER BY "Total Awarded" DESC
"""

agency_sb_regional = psql.sqldf(query, locals())

# agency_sb_regional.to_excel('Deliverables/Regional Small Business Agencies.xlsx')
agency_sb_regional

## By socio-economic status
### Top Agencies Awarding HUBZone Business (Regionally)

In [1]:
import os
import pandas as pd
import pandasql as psql


directory = "Data/Business Type"
output_file = "hubzone_regionally.csv"
processed_files_file = "hubzone_processed_regionally.txt"

names = []
months = ["October", "November", "December", "January", "February", "March", "April", "May", "June", "July", "August", "September"]
years = [2018, 2019, 2020, 2021, 2022, 2023]

for y in years:
    for m in months:
        if y == 2018 and m in ["January", "February", "March", "April", "May", "June", "July", "August", "September"]:
            continue
        if y == 2023 and m in ["October", "November", "December"]:
            continue
        names.append(f'{directory}/{m} {y}.csv')

if os.path.exists(processed_files_file):
    with open(processed_files_file, 'r') as f:
        processed_files = set(f.read().splitlines())
else:
    processed_files = set()

if os.path.exists(output_file):
    combined_results = pd.read_csv(output_file)
else:
    combined_results = pd.DataFrame()

query = """
SELECT awarding_agency_name AS "Awarding Agency", 
       historically_underutilized_business_zone_hubzone_firm AS HUBZone, 
       SUM(total_dollars_obligated) AS "Total Awarded", 
       COUNT(*) AS "# of Contracts"
FROM data
WHERE historically_underutilized_business_zone_hubzone_firm = 't'
  AND primary_place_of_performance_state_name IN ('NEBRASKA', 'IOWA', 'KANSAS', 'MISSOURI')
GROUP BY awarding_agency_name, historically_underutilized_business_zone_hubzone_firm
ORDER BY "Total Awarded" DESC
"""

for path in names:
    if path in processed_files:
        continue

    if os.path.exists(path):
        try:
            data = pd.read_csv(path, low_memory=False, on_bad_lines='skip')
            result = psql.sqldf(query, locals())
            combined_results = pd.concat([combined_results, result], ignore_index=True)
            
            combined_results.to_csv(output_file, index=False)
            with open(processed_files_file, 'a') as f:
                f.write(path + '\n')
                
        except pd.errors.ParserError as e:
            print(f"Error parsing {path}: {e}")
        except OSError as e:
            print(f"OSError while reading {path}: {e}") 

print("Processing completed.")
print(combined_results)

Processing completed.
                     Awarding Agency HUBZone  Total Awarded  # of Contracts
0              Department of Defense       t    36955412.52             124
1     Department of Veterans Affairs       t     9208518.22               6
2    General Services Administration       t     9018164.76               8
3          Department of Agriculture       t     3596019.95              12
4    Environmental Protection Agency       t     1018337.69               2
..                               ...     ...            ...             ...
629  General Services Administration       t     4498505.14              15
630  Department of Homeland Security       t     1951534.02               4
631     Department of Transportation       t      641497.41               4
632             Department of Energy       t      315415.10               2
633            Department of Justice       t       24745.68               1

[634 rows x 4 columns]


In [31]:
import pandas as pd
import pandasql as psql

typeAgencies = pd.read_csv('hubzone_regionally.csv', low_memory=False)

query="""
SELECT 
    "Awarding Agency", 
    SUM("Total Awarded") as "Total Awarded", 
    SUM("# of Contracts") AS "# of Contracts"
FROM typeAgencies
GROUP BY "Awarding Agency"
ORDER BY "Total Awarded" DESC
"""

typeAgencies = psql.sqldf(query, locals())
typeAgencies.to_excel('Deliverables/Agencies Awarding to HUBZone - Business Type (Regionally).xlsx')

### Top Agencies Awarding HUBZone Businesses (Nationally)

In [2]:
import os
import pandas as pd
import pandasql as psql


directory = "Data/Business Type"
output_file = "hubzone_nationally.csv"
processed_files_file = "hubzone_processed_nationally.txt"

names = []
months = ["October", "November", "December", "January", "February", "March", "April", "May", "June", "July", "August", "September"]
years = [2018, 2019, 2020, 2021, 2022, 2023]

for y in years:
    for m in months:
        if y == 2018 and m in ["January", "February", "March", "April", "May", "June", "July", "August", "September"]:
            continue
        if y == 2023 and m in ["October", "November", "December"]:
            continue
        names.append(f'{directory}/{m} {y}.csv')

if os.path.exists(processed_files_file):
    with open(processed_files_file, 'r') as f:
        processed_files = set(f.read().splitlines())
else:
    processed_files = set()

if os.path.exists(output_file):
    combined_results = pd.read_csv(output_file)
else:
    combined_results = pd.DataFrame()

query = """
SELECT awarding_agency_name AS "Awarding Agency", 
       historically_underutilized_business_zone_hubzone_firm AS HUBZone, 
       SUM(total_dollars_obligated) AS "Total Awarded", 
       COUNT(*) AS "# of Contracts"
FROM data
WHERE historically_underutilized_business_zone_hubzone_firm = 't'
GROUP BY awarding_agency_name, historically_underutilized_business_zone_hubzone_firm
ORDER BY "Total Awarded" DESC
"""

for path in names:
    if path in processed_files:
        continue

    if os.path.exists(path):
        try:
            data = pd.read_csv(path, low_memory=False, on_bad_lines='skip')
            result = psql.sqldf(query, locals())
            combined_results = pd.concat([combined_results, result], ignore_index=True)
            
            combined_results.to_csv(output_file, index=False)
            with open(processed_files_file, 'a') as f:
                f.write(path + '\n')
                
        except pd.errors.ParserError as e:
            print(f"Error parsing {path}: {e}")
        except OSError as e:
            print(f"OSError while reading {path}: {e}") 

print("Processing completed.")
print(combined_results)

Processing completed.
                                        Awarding Agency HUBZone  \
0                                 Department of Defense       t   
1                                Department of Commerce       t   
2                        Department of Veterans Affairs       t   
3                       Department of Homeland Security       t   
4                                   Department of State       t   
...                                                 ...     ...   
2697                            National Gallery of Art       t   
2698  U.S. International Development Finance Corpora...       t   
2699         United States Trade and Development Agency       t   
2700            Defense Nuclear Facilities Safety Board       t   
2701                        District of Columbia Courts       t   

      Total Awarded  # of Contracts  
0      2.922570e+09           10563  
1      5.606884e+08              38  
2      3.365805e+08             380  
3      2.088568e+08  

In [30]:
typeAgenciesNation = pd.read_csv('hubzone_nationally.csv', low_memory=False)

query="""
SELECT 
    "Awarding Agency", 
    SUM("Total Awarded") as "Total Awarded", 
    SUM("# of Contracts") AS "# of Contracts"
FROM typeAgenciesNation
GROUP BY "Awarding Agency"
ORDER BY "Total Awarded" DESC
"""

typeAgenciesNation = psql.sqldf(query, locals())
typeAgenciesNation.to_excel('Deliverables/Agencies Awarding to HUBZone - Business Type (Nationally).xlsx')

### Top Agencies Awarding Small Busines Businesses (Regionaly)

In [3]:
import os

directory = "Data/Business Type"
output_file = "small_business_regionally.csv"
processed_files_file = "small_business_processed_regionally.txt"

names = []
months = ["October", "November", "December", "January", "February", "March", "April", "May", "June", "July", "August", "September"]
years = [2018, 2019, 2020, 2021, 2022, 2023]

for y in years:
    for m in months:
        if y == 2018 and m in ["January", "February", "March", "April", "May", "June", "July", "August", "September"]:
            continue
        if y == 2023 and m in ["October", "November", "December"]:
            continue
        names.append(f'{directory}/{m} {y}.csv')

if os.path.exists(processed_files_file):
    with open(processed_files_file, 'r') as f:
        processed_files = set(f.read().splitlines())
else:
    processed_files = set()

if os.path.exists(output_file):
    combined_results = pd.read_csv(output_file)
else:
    combined_results = pd.DataFrame()

query = """
SELECT awarding_agency_name AS "Awarding Agency", 
       historically_underutilized_business_zone_hubzone_firm AS HUBZone, 
       SUM(total_dollars_obligated) AS "Total Awarded", 
       COUNT(*) AS "# of Contracts"
FROM data
WHERE contracting_officers_determination_of_business_size = "SMALL BUSINESS"
  AND primary_place_of_performance_state_name IN ('NEBRASKA', 'IOWA', 'KANSAS', 'MISSOURI')
GROUP BY awarding_agency_name, historically_underutilized_business_zone_hubzone_firm
ORDER BY "Total Awarded" DESC
"""

for path in names:
    if path in processed_files:
        continue

    if os.path.exists(path):
        try:
            data = pd.read_csv(path, low_memory=False, on_bad_lines='skip')
            result = psql.sqldf(query, locals())
            combined_results = pd.concat([combined_results, result], ignore_index=True)
            
            combined_results.to_csv(output_file, index=False)
            with open(processed_files_file, 'a') as f:
                f.write(path + '\n')
                
        except pd.errors.ParserError as e:
            print(f"Error parsing {path}: {e}")
        except OSError as e:
            print(f"OSError while reading {path}: {e}") 

print("Processing completed.")
print(combined_results)

Processing completed.
                           Awarding Agency HUBZone  Total Awarded  \
0                    Department of Defense       f   1.038293e+09   
1           Department of Veterans Affairs       f   1.182813e+08   
2          Department of Homeland Security       f   9.302251e+07   
3                Department of Agriculture       f   8.501465e+07   
4          General Services Administration       f   4.533316e+07   
...                                    ...     ...            ...   
1934                  Department of Energy       t   3.154151e+05   
1935               Department of Education       f   1.259510e+05   
1936  Agency for International Development       f   8.345094e+04   
1937    Consumer Product Safety Commission       f   4.125000e+04   
1938                 Department of Justice       t   2.474568e+04   

      # of Contracts  
0               4070  
1                288  
2                 45  
3                179  
4                427  
...        

In [32]:
import pandas as pd
import pandasql as psql

typeAgenciesRegionally = pd.read_csv('small_business_regionally.csv')

query="""
SELECT 
    "Awarding Agency", 
    SUM("Total Awarded") as "Total Awarded", 
    SUM("# of Contracts") AS "# of Contracts"
FROM typeAgenciesRegionally
GROUP BY "Awarding Agency"
ORDER BY "Total Awarded" DESC
"""

typeAgenciesRegionally = psql.sqldf(query, locals())
typeAgenciesRegionally.to_excel('Deliverables/Agencies Awarding to Small Businesses - Business Type (Regionally).xlsx')

### Top Agencies Awarding Small Busines Businesses (Nationally)

In [4]:
import os
import pandas as pd 
import pandasql as psql

directory = "Data/Business Type"
output_file = "small_business_nationally.csv"
processed_files_file = "small_business_processed_nationally.txt"

names = []
months = ["October", "November", "December", "January", "February", "March", "April", "May", "June", "July", "August", "September"]
years = [2018, 2019, 2020, 2021, 2022, 2023]

for y in years:
    for m in months:
        if y == 2018 and m in ["January", "February", "March", "April", "May", "June", "July", "August", "September"]:
            continue
        if y == 2023 and m in ["October", "November", "December"]:
            continue
        names.append(f'{directory}/{m} {y}.csv')

if os.path.exists(processed_files_file):
    with open(processed_files_file, 'r') as f:
        processed_files = set(f.read().splitlines())
else:
    processed_files = set()

if os.path.exists(output_file):
    combined_results = pd.read_csv(output_file)
else:
    combined_results = pd.DataFrame()

query = """
SELECT awarding_agency_name AS "Awarding Agency", 
       historically_underutilized_business_zone_hubzone_firm AS HUBZone, 
       SUM(total_dollars_obligated) AS "Total Awarded", 
       COUNT(*) AS "# of Contracts"
FROM data
WHERE contracting_officers_determination_of_business_size = "SMALL BUSINESS"
GROUP BY awarding_agency_name, historically_underutilized_business_zone_hubzone_firm
ORDER BY "Total Awarded" DESC
"""

for path in names:
    if path in processed_files:
        continue

    if os.path.exists(path):
        try:
            data = pd.read_csv(path, low_memory=False, on_bad_lines='skip')
            result = psql.sqldf(query, locals())
            combined_results = pd.concat([combined_results, result], ignore_index=True)
            
            combined_results.to_csv(output_file, index=False)
            with open(processed_files_file, 'a') as f:
                f.write(path + '\n')
                
        except pd.errors.ParserError as e:
            print(f"Error parsing {path}: {e}")
        except OSError as e:
            print(f"OSError while reading {path}: {e}") 

print("Processing completed.")
print(combined_results)

Processing completed.
                                        Awarding Agency HUBZone  \
0                                 Department of Defense       f   
1         National Aeronautics and Space Administration       f   
2                        Department of Veterans Affairs       f   
3                                 Department of Defense       t   
4               Department of Health and Human Services       f   
...                                                 ...     ...   
6336                            National Gallery of Art       t   
6337                           Selective Service System       f   
6338  U.S. International Development Finance Corpora...       t   
6339         United States Trade and Development Agency       t   
6340            Defense Nuclear Facilities Safety Board       t   

      Total Awarded  # of Contracts  
0      6.270187e+10          175649  
1      8.255011e+09             574  
2      4.768685e+09            7203  
3      2.911010e+09  

In [33]:
typeAgenciesNationally = pd.read_csv('small_business_nationally.csv')

query="""
SELECT 
    "Awarding Agency", 
    SUM("Total Awarded") as "Total Awarded", 
    SUM("# of Contracts") AS "# of Contracts"
FROM typeAgenciesNationally
GROUP BY "Awarding Agency"
ORDER BY "Total Awarded" DESC
"""

typeAgenciesNationally = psql.sqldf(query, locals())
typeAgenciesNationally.to_excel('Deliverables/Agencies Awarding to Small Businesses - Business Type (Nationally).xlsx')

typeAgenciesNationally

,Awarding Agency,Total Awarded,# of Contracts
0,Department of Defense,6.492269e+12,9274021
1,National Aeronautics and Space Administration,6.663002e+11,56373
2,Department of Homeland Security,4.190588e+11,160324
3,Department of Health and Human Services,3.908367e+11,162836
4,Department of Veterans Affairs,3.570066e+11,326081
...,...,...,...
67,Council of the Inspectors General on Integrity...,3.528457e+06,10
68,Morris K. Udall and Stewart L. Udall Foundation,2.744792e+06,27
69,Marine Mammal Commission,2.822746e+05,13
70,Administrative Conference of the U.S.,2.117173e+05,11


# Q4. Hubzone set-aside awards by NAICS for industries of Fremont businesses

In [ ]:
industry_mapping = {
    '11': 'Agriculture, Forestry, Fishing and Hunting',
    '21': 'Mining, Quarrying, and Oil and Gas Extraction',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing',
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale Trade',
    '44': 'Retail Trade',
    '45': 'Retail Trade',
    '48': 'Transportation and Warehousing',
    '49': 'Transportation and Warehousing',
    '51': 'Information',
    '52': 'Finance and Insurance',
    '53': 'Real Estate and Rental and Leasing',
    '54': 'Professional, Scientific, and Technical Services',
    '56': 'Administrative and Support and Waste Management and Remediation Services',
    '61': 'Educational Services',
    '62': 'Health Care and Social Assistance',
    '71': 'Arts, Entertainment, and Recreation',
    '72': 'Accommodation and Food Services',
    '81': 'Other Services (except Public Administration)',
    '92': 'Public Administration'
}

contracts['industry'] = contracts['naics_code'].str[:2]

contracts['industry_name'] = contracts['industry'].map(industry_mapping)

In [ ]:
import pandasql as psql

query = """
SELECT 
    industry_name AS Industry,
    naics_code AS NAICS,
    naics_description AS Description,
    type_of_set_aside AS "Set Aside",
    SUM(total_obligated_amount) AS "Total Awarded", 
    COUNT(*) AS "# of Contracts"
FROM contracts
WHERE 
    type_of_set_aside = 'HUBZone'
    AND (
        industry BETWEEN '31' AND '33' 
        OR industry IN ('21', '23', '42', '44', '45', '48', '49', '61', '62', '71', '72', '81') 
        OR industry BETWEEN '51' AND '56'
    )
GROUP BY Description, NAICS, type_of_set_aside
HAVING SUM(total_obligated_amount) > 0
ORDER BY "Total Awarded" DESC
"""

hubzone_fremont_industries = psql.sqldf(query, locals())
hubzone_fremont_industries

hubzone_fremont_industries.to_excel('Deliverables/HUBZone Set Asides by Fremont Industries.xlsx')

# Q5. Small business set-asides by NAICS for industries of Fremont businesses

In [ ]:
query = """
SELECT 
    industry_name AS Industry,
    naics_code AS NAICS,
    naics_description AS Description, 
    type_of_set_aside AS "Set Aside",
    SUM(total_obligated_amount) AS "Total Awarded", 
    COUNT(*) AS "# of Contracts"
FROM contracts
WHERE 
    type_of_set_aside = 'Small Business'
    AND (
        industry BETWEEN '31' AND '33' 
        OR industry IN ('21', '23', '42', '44', '45', '48', '49', '61', '62', '71', '72', '81') 
        OR industry BETWEEN '51' AND '56'
    )
GROUP BY Description, NAICS, type_of_set_aside
HAVING SUM(total_obligated_amount) > 0
ORDER BY "Total Awarded" DESC
"""

sb_fremont_industries = psql.sqldf(query, locals())
sb_fremont_industries

sb_fremont_industries.to_excel('Deliverables/Small Business Set Aside by Fremont Industry.xlsx')

# Q6. Profiles for any businesses listed in DSBS in Dodge County

In [ ]:
# Loading Nebraska businesses data
dsbs = pd.read_excel('Data/DSBS Dodge Business Profiles.xlsx', dtype={'NAICS, Primary and Small': str, 
                                                                      'NAICS, All (for which firm is small)': str})

In [ ]:
def fill_naics(x):
    x['naics'] = 0
    
    if pd.notna(x['NAICS, Primary and Small']):
        x['naics'] = x['NAICS, Primary and Small']

    elif pd.notna(x['NAICS, All (for which firm is small)']):
        x['naics'] = x['NAICS, All (for which firm is small)'][0:6]

    else:
        x['naics'] = 0

    return x

dsbs = dsbs.apply(fill_naics, axis = 1)

In [ ]:
dodgeBusinesses = dsbs[['Name of Firm', 'naics']]
dodgeBusinesses

dodgeBusinesses.to_excel('Deliverables/DSBS Dodge Business Profiles.xlsx')